In [ ]:
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import selenium.webdriver as webdriver
import urllib
import urllib.parse
import urllib.request
import re
import time
import os
import subprocess
from requests import get
import pandas as pd

account =  ['resortalyeska',
            'skiheavenly',
            'kirkwoodmtn',
            'mammothmountain',
            'northstar_california',
            'squawalpine',
            'sugarbowlresort',
            'arapahoe_basin',
            'aspensnowmass',
            'aspenco',
            'beavercreek',
            'breckenridgemtn',
            'coppermtn',
            'skicrestedbutte',
            'keystone_resort',
            'lovelandskiarea',
            'skipowderhorn',
            'silvertonmtn',
            'steamboatresort',
            'tellurideski',
            'vailmtn',
            'winterparkresort',
            'schweitzer_mountain',
            'sunvalley',
            'tamarackresort',
            'bigskyresort',
            'bridgerbowl',
            'mtroseskitahoe',
            'skitaos',
            'mthoodmeadows',
            'altaskiarea',
            'brightonresort',
            'deervalleyresort',
            'pcski',
            'powdermountain',
            'snowbasinresort',
            'snowbird',
            'solitudemtn',
            'jaypeakresort',
            'killingtonmtn',
            'madriverglen',
            'stowemt',
            'crystalmountain',
            'mtbakerskiarea',
            'stevenspass',           
            'grandtargheeresort',
            'jacksonhole',
            'sundanceresort',
            'bogusbasin',
            'summitatsnoqualmie',
            'skipurg',
            'brianheadresort',
            'sierra_at_tahoe',
            'skieaglecrest',
            'silvermountain',
            'skisantafe',
            'skithebeav',
            'brundagemtn',
            'skieaglepoint',
            'whitepass',
            'losttrailskiarea',
            'smugglersnotchvt',
            'redlodge',
            'monarchmountain',
            'sugarbush_vt',
            'skiwhitefish',
            'wolfcreekski',
            'mtbachelor',      
            'whistlerblackcomb',
            'revelstoke',
            'ferniealpineresort',
            'whitewaterskiresort',
            'redresort',
            'sunshinevillage',
            'kickinghorsemtn',
            'powderkingmountainresort',
            'skilouise',
            'panoramaresort',
            'castlemountainresort',
            'skibigwhite',
            'cypressmtn',
            'silverstarmtnresort',
            'sunpeaksresort',
            'mountwashington',
            'timberlinelodge',
            'skihomewood',
            'eldoramtnresort',
            'bearvalleymountainupdates',
            'diamondpeak',
            '49degreesnorth',
            'lookoutpass',
            'discoveryskiarea',
            'pebbleskiarea',
            'sunlightmountainresort',
            'skibowl',
            'dodgeridge',
            'anthonylakes',
            'marmotbasin',
            'apexmtnresort']

for i in range(0, len(account)):
    
    lastheight = 0
    
    # open the VPN-connection and wait for 60 seconds for the program to actually connect
    vpn = subprocess.Popen('c:\\Program Files\\Trust.Zone VPN Client\\trustzone_x64.exe')
    time.sleep(60)

    # collect and print the public IP address to make sure the VPN worked
    public_ip = get('https://api.ipify.org').text
    print('The public IP address is:',public_ip)
    
    # set the URL to the Instagram account
    url = 'https://www.instagram.com/' + str(account[i])

    # url: from which location are you going to scrape?
    # path: remove the basic url (everything from the first single / will be in the path)
    # folder_name: remove the / from path
    # folder_path: create a path to store the results: images/folder_name

    # make a folder to save the data
    path = urllib.parse.urlparse(url).path
    folder_name = path.replace('/', '')
    folder_path = 'images/{}/'.format(folder_name)

    # create the folder bases on folder_path
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print ('path created')

    driver = webdriver.Firefox()
    driver.get(url)
    driver.set_window_size(1000, 600000)

    # instagram shows a 'Load More' (depending on the language settings...)
    # press this button to show the complete page and scroll to the bottom
    
    # wait 10 seconds for the page to load again and continue until the page doesn't reload anymore, when
    # that happens go up a 1000 pixels in order for the page to reload and continue until it stops loading again

    clickin = driver.find_element_by_link_text('Meer laden')

    if clickin.click():
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    time.sleep(10)

    lastHeight = driver.execute_script("return document.body.scrollHeight")
    scroll_count = 0
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            scroll_count += 1
            if scroll_count <= 1:
                newHeight = lastHeight - 1000
            else:
                break
        lastHeight = newHeight
        
    only_a_and_meta_tags = SoupStrainer(['a', 'meta'])

    # parse the html of the resulting website into soup
    soup = BeautifulSoup(driver.page_source, "html.parser", parse_only = only_a_and_meta_tags)
    
    # this will close the page once scraping is completed
    driver.close()

    # scrape the number of followers for each account
    followers_account = str()

    for link in soup.find_all('meta'):
        metaprop = link.get('property')
        if metaprop == "og:description":
            description_info = link.get('content')
            followers_account_str = re.search('\d.+?(?=volgers|Followers)', str(description_info)).group(0).strip()

            # get rid of the k or K (thousands) and commas and convert to integers
            followers_account_temp = re.sub('[kK]+', '', followers_account_str)
            if followers_account_temp == followers_account_str:
                followers_account = int(float(re.sub('[,]+', '', followers_account_temp)))
            else:
                followers_account = int(float(re.sub('[,]+', '.', followers_account_temp)) * 1000)

    # scrape the links to all the images of an account
    image_url_list = []
    
    for link in soup.find_all('a'):
        if str(link.get('href')).startswith('/p'):
            m = [account[i], public_ip, "http://www.instagram.com" + str(link.get('href')), followers_account]
            image_url_list.append(m)
    
    print('The number of pictures for', account[i], 'is:', len(image_url_list))
    
    account_images = pd.DataFrame([[x[0], x[1], x[2], x[3]] for x in image_url_list])
    account_images.to_csv(os.path.join(folder_path, str(account[i]) + "_pictures.csv"), sep = ',', index = False)
    
    vpn.terminate()
    time.sleep(60)